Importing the required packages

In [16]:
import pandas as pd
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt

Loading the dataset and removing all the rows with missing values

In [17]:

# Load the dataset
df = pd.read_csv('Chicago_Traffic_Tracker_-_Historical_Congestion_Estimates_by_Segment_-_2018-Current.csv')

# Data Exploration
print(df.head())  # Displays the first few rows of the dataset
print(df.info())  # Displays column data types and non-null counts
print(df.describe())  # Displays summary statistics for numerical columns
print(df['STREET'].unique())  # Displays unique values in the 'STREET' column
print(df.shape)  # Displays the shape of the data before cleaning

# Data Cleaning: Remove rows with missing values
df_clean = df.dropna()

                     TIME  SEGMENT_ID  SPEED               STREET DIRECTION  \
0  11/20/2020 11:10:58 AM      1171.0   23.0               Harlem        SB   
1  11/20/2020 11:10:26 AM       176.0   -1.0             Garfield        WB   
2  11/20/2020 11:10:29 AM        37.0   18.0              Pulaski        NB   
3  11/20/2020 11:10:44 AM       719.0   24.0  Dr Martin L King Jr        SB   
4  11/20/2020 11:10:55 AM       495.0   -1.0               Archer        EB   

  FROM_STREET TO_STREET  LENGTH STREET_HEADING             COMMENTS  ...  \
0   North Ave   Chicago    1.01              N  Outside City Limits  ...   
1     Ashland     Damen    0.50              W                  NaN  ...   
2        31st      26th    0.50              S                  NaN  ...   
3        24th      26th    0.26              S                  NaN  ...   
4     Central   Laramie    0.51              S                  NaN  ...   

   HOUR  DAY_OF_WEEK  MONTH          RECORD_ID  START_LATITUDE  \
0 

In [18]:
df_clean1 = df_clean

In [19]:
print(df_clean1.shape)

(1130044, 22)


Checking if the SPEED column has any null values

In [20]:
print(df_clean1['SPEED'].isnull())

0          False
19         False
20         False
39         False
55         False
           ...  
9317625    False
9317632    False
9317635    False
9317643    False
9317646    False
Name: SPEED, Length: 1130044, dtype: bool


Checking if the LENGTH(Segmenth_length) column has any null values

In [21]:
print(df_clean1['LENGTH'].isnull())

0          False
19         False
20         False
39         False
55         False
           ...  
9317625    False
9317632    False
9317635    False
9317643    False
9317646    False
Name: LENGTH, Length: 1130044, dtype: bool


Checking if the SPEED column has any negative values

In [22]:
negative_values_mask = df_clean1['SPEED'] < 0
negative_values = df_clean1.loc[negative_values_mask, 'SPEED']

Considering only positive values

In [23]:
df_clean_pos = df_clean1[df_clean1['SPEED'] >= 0]

In [24]:
print(df_clean_pos['SPEED'])

0          23.0
19         26.0
20         31.0
39         21.0
55         35.0
           ... 
9317613    27.0
9317616    20.0
9317635    34.0
9317643    32.0
9317646    27.0
Name: SPEED, Length: 627073, dtype: float64


Renaming the column names to source and target

In [25]:
import numpy as np
selected_columns = ['LENGTH', 'SPEED']
df_selected = df_clean_pos[selected_columns].copy()  # Ensure to use .copy() to avoid the warning

# Renaming the selected columns
df_selected.columns = ['source', 'target']

In [26]:
print(df_selected)

         source  target
0          1.01    23.0
19         0.50    26.0
20         0.50    31.0
39         0.20    21.0
55         0.50    35.0
...         ...     ...
9317613    0.50    27.0
9317616    0.63    20.0
9317635    0.91    34.0
9317643    0.50    32.0
9317646    0.49    27.0

[627073 rows x 2 columns]


Sorting the dataframe according to the target(SPEED) column in descending order of SPEED

In [27]:
df_sorted = df_selected.sort_values(by = 'target', ascending = False)

In [28]:
print(df_sorted)

         source  target
6565068    1.54    47.0
7852696    1.54    47.0
6812102    1.54    47.0
7462370    1.54    47.0
6887168    1.54    47.0
...         ...     ...
8137929    0.25     0.0
6306208    1.00     0.0
7057138    0.56     0.0
4369233    1.00     0.0
6815159    0.25     0.0

[627073 rows x 2 columns]


Dropping the Duplicate values from both the columns

In [29]:
df_unique = df_sorted.drop_duplicates(subset=['target', 'source'])

Getting only the unique values

In [30]:
print(df_unique)

         source  target
6565068    1.54    47.0
6211402    1.23    46.0
3644527    1.54    46.0
6217433    0.50    45.0
7892615    1.02    45.0
...         ...     ...
7749664    0.49     0.0
6918286    0.43     0.0
8175651    0.91     0.0
2154339    0.63     0.0
5387893    1.23     0.0

[852 rows x 2 columns]


Restricting the dataframe to get only top 100 rows

In [31]:
df_top_100 = df_unique.head(100)

In [32]:
print(df_top_100)

         source  target
6565068    1.54    47.0
6211402    1.23    46.0
3644527    1.54    46.0
6217433    0.50    45.0
7892615    1.02    45.0
...         ...     ...
7750068    0.25    36.0
5910832    1.01    36.0
6606298    0.49    36.0
511932     0.79    36.0
744079     1.02    36.0

[100 rows x 2 columns]


Creating a new column by specifying the threshold depending on the target column values

In [38]:
def create_value_column(row):
    target = float(row['target'])
    if target > 42:
        return 0
    elif 37 <= target <= 42:
        return 1
    else:
        return 2

New column called value is created with values 0, 1 and 2 (Not Congested, Congested and Extremely Congested)

In [43]:
df_top_100.loc[:, 'value'] = df_top_100.apply(create_value_column, axis=1)

C:\Users\hpokal2\AppData\Local\Temp\ipykernel_4388\995176387.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_top_100.loc[:, 'value'] = df_top_100.apply(create_value_column, axis=1)


In [44]:
print(df_top_100)

         source  target  value
6565068    1.54    47.0      0
6211402    1.23    46.0      0
3644527    1.54    46.0      0
6217433    0.50    45.0      0
7892615    1.02    45.0      0
...         ...     ...    ...
7750068    0.25    36.0      2
5910832    1.01    36.0      2
6606298    0.49    36.0      2
511932     0.79    36.0      2
744079     1.02    36.0      2

[100 rows x 3 columns]


In [45]:
print(df_top_100.dtypes)

source    float64
target    float64
value       int64
dtype: object


Converting the dataframe to csv and downloading it to local

In [46]:
df_top_100.to_csv('sankey.csv', index = False)

### Reference: https://hemanth-110500.github.io/visualizingGeneticMutation/